In [1]:
# import libraries

import sqlite3  # for working with sqlite3 in jupyter python script
from IPython.display import Image  # for displaying images in markdown cells

In [2]:
%%html
<style>
table {align:left;display:block}  # to align html tables to left
</style> 

# Dataquest - Intermediate SQL For Data Analysis <br/> <br/> Project Title: Answering Business Questions Using SQL

## 1) Introduction and Schema Diagram

Provided by: [Dataquest.io](https://www.dataquest.io/)

In this guided project, we're going to practice using our SQL skills to answer business questions.

We'll use the Chinook database, provided as a SQLite database file called chinook.db. A copy of the database schema is below.

![title](chinook-schema.svg)

In the case where the working database **'chinook.db'** has unwanted modification when exploring this exercise, a backup database **'chinook-unmodified.db'** file is also located in this github directory folder that can be copied over the chinook.db to restore it back to its initial state.

We'll use the following code to connect our Jupyter Notebook to our database file:

In [3]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

## 2) Overview Of The Data

Let's start by getting familiar with our data. 

We can query the database to get a list of all tables and views in our database:

In [4]:
%%sql
/*
Write a query to return information on the tables and views in the database.

Write one or two queries to get familiar with the tables and to practice running SQL in this interface. Use the schema diagram on the previous screen for reference.
*/

SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## 3) Selecting Albums To Purchase

Provided by: [Dataquest.io](https://www.dataquest.io/)

The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

---
Artist Name | Genre|
--- | --- |
Regal | Hip-Hop |
Red Tone | Punk |
Meteor and the Girls | Pop |
Slim Jim Bites | Blues |

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, **so we're interested in finding out which genres sell the best in the USA.**

We need to write a query to find out which genres sell the most tracks in the USA, write up a summary of your findings, and make a recommendation for the three artists whose albums we should purchase for the store.


In [5]:
%%sql
/*
Write a query that returns each genre, with the number of tracks sold in the USA:
- in absolute numbers
- in percentages.

Write a paragraph that interprets the data and makes a recommendation for the three artists whose albums we should purchase for the store, based on sales of tracks from their genres.
*/

SELECT
    inv.billing_country AS 'Country Sold',
    SUM(il.quantity) AS 'Number of tracks sold (qty)',
    gen.name AS 'Genre Name'
FROM
    invoice AS inv
LEFT JOIN
    invoice_line AS il
    ON il.invoice_id = inv.invoice_id
LEFT JOIN 
    track AS tra
    ON tra.track_id = il.track_id
LEFT JOIN
    genre AS gen
    ON gen.genre_id = tra.genre_id
WHERE
    inv.billing_country = 'USA'
    AND
    (gen.name LIKE '%hip%'
     OR
     gen.name LIKE '%punk%'
     OR
     gen.name LIKE '%pop%'
     OR
     gen.name LIKE '%blues%')
GROUP BY
    gen.name
ORDER BY
    2 DESC;

 * sqlite:///chinook.db
Done.


Country Sold,Number of tracks sold (qty),Genre Name
USA,130,Alternative & Punk
USA,36,Blues
USA,22,Pop
USA,20,Hip Hop/Rap


#### Findings (Selecting Albums To Purchase):
Based on the shortlisted findings above based on genre popularity measured by number of tracks sold, it would appear the below artist/genre are preferred in descending order, where **'Alternative & Punk' genre dominates the sales chart by far**, while the other genres have less of a difference among them.

Rank |Artist Name | Artist Genre
--- | --- | --- |
1 | Red Tone | Punk | 
2 | Slim Jim Bites | Blues |
3 | Meteor and the Girls | Pop | 
4 | Regal | Hip-Hop |

## 4) Analysing Employee Sales Performance

Provided by: [Dataquest.io](https://www.dataquest.io/)

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

You might like to consider whether any extra columns from the employee table explain any variance you see, or whether the variance might instead be indicative of employee performance.

In [6]:
%%sql
/*
Write a query that finds the total dollar amount of sales assigned to each sales support agent within the company. Add any extra attributes for that employee that you find are relevant to the analysis.

Write a short statement describing your results, and providing a possible interpretation.
*/

SELECT
    emp.first_name || ' ' || emp.last_name AS employee_name,
    SUM(inv.total) AS invoice_total,
    emp.*
FROM
    employee AS emp
LEFT JOIN
    customer AS cus
    ON cus.support_rep_id = emp.employee_id
LEFT JOIN
    invoice AS inv
    ON inv.invoice_id= cus.customer_id
WHERE
    emp.title = 'Sales Support Agent'
GROUP BY
    emp.employee_id
ORDER BY
    invoice_total DESC;

 * sqlite:///chinook.db
Done.


employee_name,invoice_total,employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
Jane Peacock,165.32999999999998,3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
Margaret Park,155.43,4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
Steve Johnson,143.55,5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


#### Findings (Analysing Employee Sales Performance):
The top rankings in terms of employee sales as such.

There do not seem to have any observable trends within the employee data itself that differentiates their sales performance.

As such, the sales performance could be due to other factors such as:
- genre of tracks (could each sales representative specialise in certain genre?)
- country where tracks are sold (could each sales representative specialise in customers or outlets from specific countries?)
- single track or tracks in albums (could each sales representative specialise in singles / album tracks?)

Rank |employee_name | invoice_total
--- | --- | --- |
1 | Jane Peacock | 165.33 | 
2 | Margaret Park | 155.43 |
3 | Steve Johnson | 143.55 | 

## 5) Analysing Sales By Country

Provided by: [Dataquest.io](https://www.dataquest.io/)

Your next task is to analyze the sales data for customers from each different country. You have been given guidance to use the country value from the customers table, and ignore the country from the billing address in the invoice table.

In particular, you have been directed to calculate data, for each country, on the:
- total number of customers
- total value of sales
- average value of sales per customer
- average order value

Because there are a number of countries with only one customer, you should group these customers as "Other" in your analysis.

---
**Tip:**
You can use the following 'trick' to force the ordering of "Other" to last in your analysis.

If there is a particular value that you would like to force to the top or bottom of results, you can implement a sorting feature in a subquery:
- Put what would normally be your most outer query in a subquery with a case statement that adds a numeric column, and then in the outer query sort by that column.

In [7]:
%%sql
/*
Write a query that collates data on purchases from different countries.

- Where a country has only one customer, collect them into an "Other" group.

- The results should be sorted by the total sales from highest to lowest, with the "Other" group at the very bottom.

- For each country, include:
    - total number of customers
    - total value of sales
    - average value of sales per customer
    - average order value
*/


/*
sub1: create subquery to extract the key variables needed grouped by country
*/

WITH sub1 AS
            (
            SELECT
                cus.country,
                COUNT(DISTINCT cus.customer_id) AS total_number_of_customers,
                SUM(inv.total) AS total_value_of_sales,
                SUM(inv.total) / COUNT(DISTINCT cus.customer_id) AS average_value_of_sales_per_customer,
                AVG(inv.total) AS average_order_value
            FROM
                customer AS cus
            LEFT JOIN
                invoice AS inv
                ON inv.customer_id = cus.customer_id
            GROUP BY
                cus.country
            ORDER BY
                3 DESC
            ),

    
/*
sub3: create subquery from sub1 to group by countries where only 1 customer to new country variable using CASE statements
*/

    sub3 AS (
            SELECT
                sub2.country_modified AS country,
                SUM(sub2.total_number_of_customers) AS total_number_of_customers,
                SUM(sub2.total_value_of_sales) AS total_value_of_sales,
                AVG(sub2.average_value_of_sales_per_customer) AS average_value_of_sales_per_customer,
                AVG(sub2.average_order_value) AS average_order_value
            FROM
                (
                SELECT
                    sub1.*,
                    CASE
                        WHEN sub1.total_number_of_customers = 1 THEN 'Others'
                        ELSE sub1.country
                    END AS country_modified
                FROM
                    sub1
                ) AS sub2
            GROUP BY
                sub2.country_modified
            )

/*
sub4: create subquery from sub3 to create new sort variable for country labeled as 'Others' using CASE statements so that it can be sorted to bottom.
THEN select the relevant variables for final query.
*/
    
SELECT
    sub4.country AS country,
    sub4.total_number_of_customers AS total_number_of_customers,
    sub4.total_value_of_sales AS total_value_of_sales,
    sub4.average_value_of_sales_per_customer AS average_value_of_sales_per_customer,
    sub4.average_order_value AS average_order_value
FROM
    (
    SELECT
        sub3.*,
        CASE
            WHEN sub3.country = 'Others' THEN 1
            ELSE 0
        END AS sort
    FROM
        sub3
    ) AS sub4
ORDER BY
    sub4.sort ASC,
    sub4.total_value_of_sales DESC;

 * sqlite:///chinook.db
Done.


country,total_number_of_customers,total_value_of_sales,average_value_of_sales_per_customer,average_order_value
USA,13,1040.4900000000005,80.03769230769234,7.942671755725194
Canada,8,535.59,66.94875,7.047236842105264
Brazil,5,427.6799999999999,85.53599999999997,7.011147540983605
France,5,389.07000000000005,77.81400000000001,7.781400000000001
Germany,4,334.61999999999995,83.65499999999999,8.161463414634145
Czech Republic,2,273.23999999999995,136.61999999999998,9.107999999999999
United Kingdom,3,245.52,81.84,8.768571428571429
Portugal,2,185.13,92.565,6.3837931034482756
India,2,183.14999999999995,91.57499999999997,8.72142857142857
Others,15,1094.9399999999998,72.996,7.445071062271063


#### Findings (Analysing Sales By Country):

**USA is the country with top sales value and customers**, with pretty good average sales value per customer, and average order value as well.

## 6) Albums VS Individual Tracks

Provided by: [Dataquest.io](https://www.dataquest.io/)

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:
- purchase a whole album
- purchase a collection of one or more individual tracks.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

It is very common when you are performing an analysis to have 'edge cases' which prevent you from getting a 100% accurate answer to your question. In this instance, we have two edge cases to consider:
- Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.
- Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.

In the first case, since our analysis is concerned with maximizing revenue we can safely ignore albums consisting of only a few tracks. The company has previously done analysis to confirm that the second case does not happen often, so we can ignore this case also.

In order to answer the question, we're going to have to identify whether each invoice has all the tracks from an album. We can do this by getting the list of tracks from an invoice and comparing it to the list of tracks from an album. We can find the album to compare the purchase to by looking up the album that one of the purchased tracks belongs to. It doesn't matter which track we pick, since if it's an album purchase, that album will be the same for all tracks.

In [8]:
%%sql

/*
Write a query that categorizes each invoice as either an album purchase or not, and calculates the following summary statistics:  
- Number of invoices
- Percentage of invoices

Write one to two sentences explaining your findings, and making a prospective recommendation on whether the Chinook store should continue to buy full albums from record companies    
*/


/*
First exploring the transformation workings, review how the joined tables looks like without any group by functions
*/

SELECT
    inv.invoice_id,
    il.track_id,
    alb.album_id,
    tra.name AS track_name,
    alb.title AS album_title
FROM
    invoice AS inv
LEFT JOIN
    invoice_line AS il
    ON il.invoice_id = inv.invoice_id
LEFT JOIN
    track AS tra
    ON tra.track_id = il.track_id
LEFT JOIN
    album AS alb
    ON alb.album_id = tra.album_id
ORDER BY
    inv.invoice_id
LIMIT 5;


 * sqlite:///chinook.db
Done.


invoice_id,track_id,album_id,track_name,album_title
1,1158,91,Right Next Door to Hell,Use Your Illusion I
1,1159,91,Dust N' Bones,Use Your Illusion I
1,1160,91,Live and Let Die,Use Your Illusion I
1,1161,91,Don't Cry (Original),Use Your Illusion I
1,1162,91,Perfect Crime,Use Your Illusion I


In [9]:
%%sql

/*
Now Group By each invoice, and count distinct invoice ID and albums ID to see how many unique albums per invoice

sub1: create subquery for table of invoices with only 1 album sales
    
sub2: create subquery for table of invoices with more than 1 album sales with 'EXCEPT' clause to SELECT non-overlapping rows with sub1 (for checking purposes)
    
sub3: use CASE statements to create new attribute to label album or non-album sales by matching against the filtered invoices in defined subqueries
*/

WITH sub1 AS
            (
            SELECT
                inv.invoice_id
            FROM
                invoice AS inv
            LEFT JOIN
                invoice_line AS il
                ON il.invoice_id = inv.invoice_id
            LEFT JOIN
                track AS tra
                ON tra.track_id = il.track_id
            LEFT JOIN
                album AS alb
                ON alb.album_id = tra.album_id
            GROUP BY
                inv.invoice_id
            HAVING
                COUNT(DISTINCT alb.album_id) = 1
            ORDER BY
                inv.invoice_id
            ),

     sub2 AS
            (
            SELECT
                inv.invoice_id AS invoice_id
            FROM
                invoice AS inv

            EXCEPT

            SELECT
                sub1.invoice_id AS invoice_id
            FROM
                sub1
            ),
            
     sub3 AS
            (     
            SELECT
                inv.invoice_id,
                CASE WHEN inv.invoice_id in (SELECT sub1.invoice_id AS invoice_id
                                             FROM sub1)
                                             THEN 'album sales'
                                             ELSE 'non-album sales'
                                             END AS album_sales
            FROM
                invoice AS inv
            LEFT JOIN
                invoice_line AS il
                ON il.invoice_id = inv.invoice_id
            LEFT JOIN
                track AS tra
                ON tra.track_id = il.track_id
            LEFT JOIN
                album AS alb
                ON alb.album_id = tra.album_id
            GROUP BY
                inv.invoice_id
            ),
        
     sub4 AS
            (
            SELECT
                album_sales,
                COUNT(invoice_id) AS number_of_invoices
            FROM
                sub3
            GROUP BY
                album_sales       
            )
       
        
/*
use OVER() clause to compute SUM() over each row
*/

SELECT
    album_sales AS 'Album Sales',
    number_of_invoices AS 'Number of invoices',
    ROUND(CAST(number_of_invoices AS Float) / CAST(SUM(number_of_invoices) OVER() AS Float) * 100, 1)
    AS 'Percentage of invoices (%)'
FROM
    sub4
GROUP BY
    album_sales;

 * sqlite:///chinook.db
Done.


Album Sales,Number of invoices,Percentage of invoices (%)
album sales,171,27.9
non-album sales,443,72.1


#### Findings (Albums VS Individual Tracks):

Based on the above findings, **non-album sales constitute the majority of invoice sales based on invoice numbers.**

Suggest further analysis can be made on whether the Chinook store should continue to buy full albums from record companies:
- Eg. Album/non-album sales analysis by other attributes such as total monetary value.


## 7) Conclusion

#### Example SQL skills:
- Subqueries
- Multiple joins (to filter columns)
- Set operations (example of set operations UNION, INTERSECT, EXCEPT to filter rows)
- Aggregate functions

#### Potential areas for further analysis
- Which artist is used in the most playlists?
- How many tracks have been purchased vs not purchased?
- Is the range of tracks in the store reflective of their sales popularity?
- Do protected vs non-protected media types have an effect on popularity?